In [1]:
import sys
sys.path.append("../src")

# If you do some changes in file. jupiter notebook will update saved files 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
#name file of doccano and explanation maps 
rational_path='rationale'

In [3]:
import captum.attr as a
from torch.utils.data import DataLoader

from dataset import OurDataset
from evaluate import EvaluateExplanation
from explain import STS_ExplainWrapper, ExplanationExecuter
from compare_docano_XAI import Compare_docano_XAI,Check_docano_XAI, Hyper_optimalization


In [4]:
dataset = OurDataset(csv_dirpath="./data")

In [5]:
file='rationale'


In [ ]:
check=Check_docano_XAI(rationale_dataset_path=f'../data/annotations/{file}.json',xai_dataset=dataset,tuple_indexes=dataset.fact_check_post_mapping) #data=doccano importance_map=xai
indexes_doc, indexes_xai, doc_data=check.get_matched_doccano()

In [ ]:
length_dataset=round((0.2/100)*len(indexes_xai))
indexes_xai=indexes_xai[:length_dataset]    
dataset.fact_check_post_mapping = [dataset.fact_check_post_mapping[i] for i in indexes_xai]
# dataset.fact_check_post_mapping = dataset.fact_check_post_mapping[10:100] #smaller ds for testing purposes
loader = DataLoader(dataset, batch_size=1)

In [8]:
#### layer method ####
model = STS_ExplainWrapper.setup_t5_transformer("../models/GTR-T5-FT")

In [ ]:
#### ordinary method ####
model = STS_ExplainWrapper.setup_t5_transformer("../models/GTR-T5-FT",interpretable_embeddings=True)

c:\Users\Dell\anaconda3\envs\projectXAI\lib\site-packages\captum\attr\_models\base.py:191: UserWarning: In order to make embedding layers more interpretable they will be replaced with an interpretable embedding layer which wraps the original embedding layer and takes word embedding vectors as inputs of the forward function. This allows us to generate baselines for word embeddings and compute attributions for each embedding dimension. The original embedding layer must be set back by calling `remove_interpretable_embedding_layer` function after model interpretation is finished. 
  warnings.warn(


In [ ]:
# method = a.LayerGradientXActivation(model, layer=model.get_embedding_layer())
method = a.InputXGradient(model)
explain = ExplanationExecuter(method, compute_baseline=False, visualize_explanation=False)

If you dont have explanation maps

In [11]:
evaluation = EvaluateExplanation(rationale_path=f"./tmp/{rational_path}.json", verbose=True)
final_metric, all_metrics = evaluation.evaluate(loader, explain)


  0%|          | 0/1 [00:00<?, ?it/s]c:\Users\Dell\anaconda3\envs\projectXAI\lib\site-packages\captum\_utils\gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
100%|██████████| 1/1 [06:03<00:00, 363.17s/it]


Computing metric: aopc_suff


  0%|          | 0/1 [00:07<?, ?it/s]


KeyboardInterrupt: 

In [13]:
all_metrics

{'faithfulness': {}, 'plausibility': {'auprc_plau': 0.550458665776}}

If you have explanation maps

In [ ]:
method='Saliency'
normalization='without_normalize'
method_param={'Saliency':{'parameters': {'abs': False}}}
model_param={}
explanations=[]

explanations=Hyper_optimalization.load_explanations(f'./tmp/test_t5.json',method_param,model_param,method,explanations)
comparing=Compare_docano_XAI(docano_dataset=doc_data,importance_map=explanations)
explanations=comparing.change_token_explanation_to_sentence(model,dataset)

In [ ]:
# Need to put all methods from text file manually.
# Also doesn't matter what method will be put into ExplanationExecuter 

evaluation = EvaluateExplanation(verbose=True,rationale_path=f'../data/annotations/{file}.json')
final_metric, all_metrics = evaluation.evaluate(loader, explain,explanation_maps=explanations,method_name='Input X Gradient')  #tables

Computing metric: auprc_plau


 90%|█████████ | 9/10 [00:00<00:00, 14.25it/s]


Computing metric: token_f1_plau


 90%|█████████ | 9/10 [00:00<00:00, 15.76it/s]


Computing metric: token_iou_plau


 90%|█████████ | 9/10 [00:00<00:00, 14.68it/s]
c:\Users\Dell\Desktop\Kinit\AutoXAI\notebooks\../src\evaluate.py:262: RuntimeWarning: Mean of empty slice.
  faithfullness = np.array(list(all_metrics["faithfulness"].values())).mean()
c:\Users\Dell\anaconda3\envs\projectXAI\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
final_metric

0.2615896462751331

In [ ]:
all_metrics

{'faithfulness': {'aopc_compr': 0.07188656, 'aopc_suff': 0.934923},
 'plausibility': {'auprc_plau': 0.05932348928028465,
  'token_f1_plau': 0.0,
  'token_iou_plau': 0.0}}